In [1]:
import os
import json
from copy import copy
import numpy

In [2]:
mode = 'ivectors'
class Data:
    def __init__(self, segment, oracle_models):
        self.x = segment[mode][0]['value']
        oracle_speakers = [oracle_model['speaker_id'] for oracle_model in oracle_models]
        for oracle_model in oracle_models:
            self.x  = numpy.concatenate((self.x, oracle_model[mode][0]))
        self.y = numpy.asarray([oracle_speaker == segment['speakers'][0]['speaker_id'] for oracle_speaker in oracle_speakers], dtype = float)

In [3]:
folder = 'exp/json'
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
data = []
for f in files:
    # Getting data from json
    f = open(os.path.join(folder, f))
    recording = json.loads(f.readline())
    # Removing multiple-speaker segments
    recording['segments'] = [segment for segment in recording['segments'] if len(segment['speakers']) == 1]
    # Removing extra speakers
    recording['segments'] = [segment for segment in recording['segments'] if segment['speakers'][0]['speaker_id'] in ['A', 'B']]
    # Dealing with multiple-vector segments and generating IDs
    new_segments = []
    new_segments_count = 0
    for segment in recording['segments']:
        for i in range(len(segment['ivectors'])):
            new_segment = copy(segment)
            new_segment['segment_id'] = str(new_segments_count).zfill(3)
            new_segments_count += 1
            new_segment['ivectors'] = [segment['ivectors'][i]]
            new_segment['xvectors'] = [segment['xvectors'][i]]
            new_segments.append(new_segment)
    recording['segments'] = new_segments
    # Lists to Numpy arrays
    for segment in recording['segments']:
        segment['ivectors'][0]['value'] = numpy.asarray(segment['ivectors'][0]['value'])
        segment['xvectors'][0]['value'] = numpy.asarray(segment['xvectors'][0]['value'])
    # Getting speakers list
    speakers = list(set([segment['speakers'][0]['speaker_id'] for segment in recording['segments']]))
    if len(speakers) == 2:
        # Getting speakers oracle models
        oracle_limit = 5
        oracle_segments = [[segment for segment in recording['segments'] if segment['speakers'][0]['speaker_id'] == speaker][:oracle_limit] for speaker in speakers]
        oracle_models = []
        for n, speaker_n in enumerate(speakers):
            ivector_n = oracle_segments[n][0]['ivectors'][0]['value']
            xvector_n = oracle_segments[n][0]['xvectors'][0]['value']
            for segment in oracle_segments[n][1:]:
                ivector_n = ivector_n + segment['ivectors'][0]['value']
                xvector_n = xvector_n + segment['xvectors'][0]['value']
            ivector_n = ivector_n / len(oracle_segments[n])
            xvector_n = xvector_n / len(oracle_segments[n])
            oracle_models.append({ 'speaker_id': speaker_n, 'ivectors': [ivector_n], 'xvectors': [xvector_n] })
        # Generating dataset
        for segment in recording['segments']:
            data.append(Data(segment, oracle_models))

In [23]:
'''import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc0 = nn.Linear(128 * 3, 384)
        self.fc1 = nn.Linear(384, 384)
        self.fc2 = nn.Linear(384, 384)
        self.fc3 = nn.Linear(384, 192)
        self.fc4 = nn.Linear(192, 192)
        self.fc5 = nn.Linear(192, 192)
        self.fc6 = nn.Linear(192, 96)
        self.fc7 = nn.Linear(96, 2)
    def forward(self, x):
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        x = F.log_softmax(x, dim = 1)
        return x'''

In [25]:
'''import torch.optim as optim

net = Net().cuda()

optimizer = optim.Adam(net.parameters(), lr = 0.0001)

def custom_loss(input, target):
    return ((input - target) ** 2).sum() / input.data.nelement()

epochs = 10
for epoch in range(epochs):
    for entry in data:
        x = torch.FloatTensor(entry.x).cuda()
        y = torch.FloatTensor(entry.y).cuda()
        net.zero_grad()
        output = net(x.view(-1, 128 * 3))
        loss = custom_loss(output, y)
        loss.backward()
        optimizer.step()
    print(loss)'''

tensor(1.5870, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5956, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5598, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5599, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.6470, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5598, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5597, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5598, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5598, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.5597, device='cuda:0', grad_fn=<DivBackward0>)
